In [1]:
import torch

print("PyTorch Version:", torch.__version__)
print("MPS Available:", torch.backends.mps.is_available())

PyTorch Version: 2.2.2
MPS Available: True


In [2]:
import os

# Define paths
BASE_DIR = "Dataset/kaggle7"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")

print("Training Data Path:", TRAIN_DIR)
print("Testing Data Path:", TEST_DIR)

Training Data Path: Dataset/kaggle7/train
Testing Data Path: Dataset/kaggle7/test


In [3]:
##Library
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import cv2
from PIL import Image

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

/Users/sishengliang/opt/anaconda3/envs/happiness-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Emotion to happiness score mapping
emotion_to_score = {
    "happy": 9,
    "neutral": 6,
    "surprise": 7,
    "angry": 2,
    "sad": 2,
    "fear": 3,
    "disgust": 1,
}

In [5]:
# Define transformations (resize, normalize, augment)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load dataset using ImageFolder
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=transform)
test_dataset = datasets.ImageFolder(root=TEST_DIR, transform=transform)

# Convert labels to happiness scores
train_dataset.targets = [emotion_to_score[train_dataset.classes[label]] for label in train_dataset.targets]
test_dataset.targets = [emotion_to_score[test_dataset.classes[label]] for label in test_dataset.targets]

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print class labels detected
print("Class Labels Detected:", train_dataset.classes)

# Print dataset details
print("✅ Data Loaded Successfully!")
print(f"➡️ Training Images: {len(train_dataset)}")
print(f"➡️ Testing Images: {len(test_dataset)}")
print(f"➡️ Class Labels: {train_dataset.classes}")



Class Labels Detected: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
✅ Data Loaded Successfully!
➡️ Training Images: 28709
➡️ Testing Images: 7178
➡️ Class Labels: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [ ]:
# Visualize one batch
images, scores = next(iter(train_loader))
image = images[0].permute(1, 2, 0).numpy()

plt.imshow(image)
plt.title(f"Happiness Score: {scores[0].item()}")
plt.axis("off")
plt.show()

In [6]:
import torchvision.models as models

# Define Model
class HappinessModel(nn.Module):
    def __init__(self):
        super(HappinessModel, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Linear(2048, 1)  # Regression output (1 number)

    def forward(self, x):
        return self.model(x)

# Load model to GPU (if available)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = HappinessModel().to(device)

/Users/sishengliang/opt/anaconda3/envs/happiness-env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sishengliang/opt/anaconda3/envs/happiness-env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/sishengliang/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:11<00:00, 9.09MB/s]


In [ ]:
import torch.optim as optim

# Define loss function & optimizer
criterion = nn.MSELoss()  # Regression loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
print("🚀 Training Started...")
for epoch in range(epochs):
    total_loss = 0
    for images, scores in train_loader:
        images, scores = images.to(device), scores.to(device).float()

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, scores)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f" Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

# Save trained model
torch.save(model.state_dict(), "happiness_model.pth")
print("🎉 Training complete! Model saved.")

🚀 Training Started...


KeyboardInterrupt: 

In [ ]:
from PIL import Image
import cv2

# Load Model
def load_model():
    model = HappinessModel()
    model.load_state_dict(torch.load("happiness_model.pth", map_location=torch.device("cpu")))
    model.eval()
    return model

# Image Preprocessing
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

def predict_happiness(image_path):
    model = load_model()
    
    image = Image.open(image_path).convert("RGB")
    image = transform_test(image).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        score = model(image).item()

    return round(score, 2)

# Test on an image
test_image = "Dataset/kaggle7/test/happy/1.jpg"  # Replace with actual test image
print("Predicted Happiness Score:", predict_happiness(test_image))

In [ ]:
import streamlit as st
from PIL import Image

st.title("Happiness Score Detector")

uploaded_file = st.file_uploader("Upload a group photo", type=["jpg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    score = predict_happiness(uploaded_file)
    
    st.image(image, caption="Uploaded Image", use_column_width=True)
    st.write(f"Predicted Happiness Score: {score}/10")